In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import os

In [2]:
# 获取当前路径下的所有.nc文件名
current_path = os.getcwd()
file_names = [file for file in os.listdir(current_path) if file.endswith('.nc')]

# 初始化空 DataFrame，用于存储所有月份数据
result_df = pd.DataFrame()
for k in range(len(file_names)):
    
    # 打开NetCDF文件，返回一个Dataset对象，类似于字典结构，包含了文件中的所有数据和元数据
    ds = xr.open_dataset(file_names[k], engine='netcdf4', decode_times=False)
    varible_name = file_names[k].split('.')[0]
    subset_data = ds[varible_name][-3, :, :] #2018年的经纬度截面，整个数据集的起始时间是1925年
    
    # 获取非nan值的位置索引
    # np.where返回一个元组，包含两个数组：第一个是纬度索引，第二个是经度索引
    non_nan_lat, non_nan_lon = np.where(~np.isnan(subset_data.values) & (subset_data.values != 0))
    
    # 获取非NaN且非0值的实际数据
    non_nan_values = subset_data.values[non_nan_lat, non_nan_lon]
    
    # 构建包含经纬度和对应值的DataFrame
    df_non_nan = pd.DataFrame({
        'lat': subset_data.lat.values[non_nan_lat],  # 转换为真实纬度值
        'lon': subset_data.lon.values[non_nan_lon],  # 转换为真实经度值
        f'{varible_name}': non_nan_values  # 非NaN且非0的实际值
    })

    
    # 合并数据：首次循环直接赋值，后续按经纬度关联合并
    if result_df.empty:
        result_df = df_non_nan
    else:
        # 按经纬度匹配合并，确保同一经纬度的月份数据对应
        result_df = result_df.merge(df_non_nan, on=['lat', 'lon'], how='outer')

ds.close()

In [3]:
# #保存数据文件（把大DataFrame拆分并分别写入不同工作表）
# max_rows_per_sheet = 500000  
# num_sheets = len(result_df) // max_rows_per_sheet + (1 if len(result_df) % max_rows_per_sheet != 0 else 0)

# with pd.ExcelWriter('./全球人工氮肥施用量(2018).xlsx') as writer:
#     for i in range(num_sheets):
#         start_idx = i * max_rows_per_sheet
#         end_idx = (i + 1) * max_rows_per_sheet
#         sub_df = result_df.iloc[start_idx:end_idx]
#         sub_df.to_excel(writer, sheet_name=f'Sheet_{i + 1}', index=False)

# 保存文件(数据文件行列超出excel的范围，用csv保存)
result_df.to_csv('./全球人工氮肥施用量(2018).csv', index=False)

In [4]:
result_df.shape

(2248830, 12)